In [1]:
import requests


In [2]:
def get_data(imdbdId):
    try:

        url = "https://caching.graphql.imdb.com/"

        payload = {
            "query": 'query TitleRatingsSubPage($tconst: ID!) {\n  title(id: $tconst) {\n    id\n    ...TitlePageMetaData\n    ...TitleSeasonsInfo\n    canRate {\n      isRatable\n      __typename\n    }\n    \n    __typename\n  }\n}\nfragment TitlePageMetaData on Title {\n  id\n  runtime {\n    seconds\n    __typename\n  }\n  ratingsSummary {\n    aggregateRating\n    voteCount\n  }  \n  titleGenres {\n    genres(limit: 3) {\n      genre {\n        text\n      }\n      __typename    }\n  } countriesOfOrigin{ \n countries(limit: 1) { id }__typename}                        \n spokenLanguages{ \n spokenLanguages(limit: 1) { id }__typename}   storylineKeywords: keywords(first:100) { edges { node { legacyId text } } total }  \n  titleType {\n    id\n    text\n    canHaveEpisodes\n    isSeries\n    isEpisode\n    __typename\n  }\n  titleText {\n    text\n    __typename\n  }\n  originalTitleText {\n    text\n    __typename\n  }\n  releaseYear {\n    year\n    endYear\n    __typename\n  }\n  plot {\n    plotText {\n      plainText\n      __typename\n    }\n    __typename\n  }\n  primaryImage {\n    url\n    height\n    width\n    caption {\n      plainText\n      __typename\n    }\n    __typename\n  }\n  series {\n    episodeNumber {\n      episodeNumber\n      seasonNumber\n      __typename\n    }\n    nextEpisode {\n      id\n      __typename\n    }\n    previousEpisode {\n      id\n      __typename\n    }\n    series {\n      id\n      titleText {\n        text\n        __typename\n      }\n      originalTitleText {\n        text\n        __typename\n      }\n      titleType {\n        canHaveEpisodes\n        __typename\n      }\n      releaseYear {\n        year\n        endYear\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  canHaveEpisodes\n  castPageTitle: credits(first: 4, filter: {categories: [\n"cast"]}) {\n    edges {\n      node {\n        name {\n          id\n          nameText {\n            text\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  creatorsPageTitle: principalCredits(filter: {categories: [\n"creator"]}) {\n    ...CreditNamePageDescription\n    __typename\n  }\n  directorsPageTitle: principalCredits(filter: {categories: [\n"director"]}) {\n    ...CreditNamePageDescription\n    __typename\n  }\n}\nfragment CreditNamePageDescription on PrincipalCreditsForCategory {\n  credits(limit: null) {\n    name {\n      id\n      nameText {\n        text\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\nfragment TitleSeasonsInfo on Title {\n  totalEpisodes: episodes {\n    unknownFilter: displayableSeasons(last: 1) {\n      edges {\n        node {\n          season\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    seasonIndex: displayableSeasons(first: 15) {\n      edges {\n        node {\n          season\n        }\n      }\n      total\n      __typename\n    }\n    episodes(first: 10) {\n      total\n      __typename\n    }\n    __typename\n  }\n}',
            "operationName": "TitleRatingsSubPage",
            "variables": {"locale": "en-US", "tconst": imdbdId},
        }
        headers = {"Content-Type": "application/json"}

        response = requests.request("POST", url, json=payload, headers=headers)
        if response.status_code == 200:
            return response.json()
        else:
            print("Invalid request: ", response.status_code)

    except Exception as err:
        print(err)

In [3]:
def counter(increment=False):
    with open("couter.txt", "r") as f:
        couterval = int(f.readline())
        if increment:
            with open("couter.txt", "w") as f2:
                f2.write(str(couterval + 1))
        return couterval

In [4]:
import pandas as pd
def get_imdb_id(index):
    df = pd.read_csv("nonp.csv")
    return df["id"][index]

In [6]:
print(get_imdb_id(1))

tt0011237


In [7]:
import json
def write_data(df):
    response = None
    try:
        index = counter()
        imdbid = get_imdb_id(index)

        response = get_data(imdbid)

        spoken_langs = "en"
        cast = []
        crew = []
        country = "US"
        keywords = []
        votercount = 0
        try:
            keywords = json.dumps(
                [
                    keyword["node"]["text"]
                    for keyword in response["data"]["title"]["storylineKeywords"][
                        "edges"
                    ]
                ]
            )
        except:pass
        try:
            spoken_langs = (
                response["data"]["title"]["spokenLanguages"]["spokenLanguages"][0][
                    "id"
                ],
            )
        except:pass
        try:
            cast = json.dumps(
                [
                    actor["node"]["name"]["id"]
                    for actor in response["data"]["title"]["castPageTitle"]["edges"]
                ]
            )
        except:pass
        try:

            crew = json.dumps(
                [
                    actor["name"]["id"]
                    for credits in response["data"]["title"]["directorsPageTitle"]
                    for actor in credits["credits"]
                ]
                + [
                    actor["name"]["id"]
                    for credits in response["data"]["title"]["creatorsPageTitle"]
                    for actor in credits["credits"]
                ]
            )
        except:pass
        try:
            country = response["data"]["title"]["countriesOfOrigin"]["countries"][0][
                "id"
            ]
        except:pass
        try:
            votercount = response["data"]["title"]["ratingsSummary"]["voteCount"]
        except:
            pass
        imageUrl = ""
        try:
            imageUrl = response["data"]["title"]["primaryImage"]["url"]
        except:
            pass
        rating = 5
        try:
            rating = response["data"]["title"]["ratingsSummary"]["aggregateRating"]
        except:
            pass
        plot = ""
        try:
            plot= response["data"]["title"]["plot"]["plotText"]["plainText"]
        except:
            pass

        new_row = {
            "id": response["data"]["title"]["id"],
            "imageUrl": imageUrl,
            "rating": rating,
            "voteCount": votercount,
            "country": country,
            "language": spoken_langs,
            "keywords": keywords,
            "plot": plot,
            "cast": cast,
            "crew": crew,
        }
        print(True)

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

        counter(True)
        return df
    except Exception as err:
        print(err)
        print(response)

In [11]:
df = pd.read_csv("imdbData5934to7500.csv")
for _ in range(1300):

    df = write_data(df)
try:
    if df.shape:
        df.to_csv("imdbData5934to7500.csv", index=False)
except:
    pass

True


C:\Users\nithi\AppData\Local\Temp\ipykernel_21516\1225794537.py:95: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
